In [1]:
%cd ..
%echo %cd%

d:\Repos\example
d:\Repos\example


In [2]:
import hydra
from omegaconf import DictConfig, OmegaConf
import json
from pathlib import Path
import numpy as np
import torch

In [ ]:
# e.g. load a config
with hydra.initialize(version_base=None, config_path="../config", job_name="test_app"):
    cfg = hydra.compose(config_name="train_model.yaml", overrides=[])

# e.g. print the config
print(OmegaConf.to_yaml(cfg))

: 

: 

In [ ]:
# e.g. instantiate dataloaders
train_loader = hydra.utils.instantiate(cfg.data.dataloaders.train)
test_loader = hydra.utils.instantiate(cfg.data.dataloaders.test)

# e.g. print first batch of train_loader
for batch in train_loader:
    print(batch)
    break

: 

In [7]:
from modules.utils.aggregator import load_folders, load_files
import pandas as pd


In [4]:
load_files(r"./data\models\mnist_net2_basic_0")

{'_directory': './data\\models\\mnist_net2_basic_0',
 'config': {'seed': 0,
  'suffix': '',
  'name': 'mnist_net2_basic_0',
  'save_dir': './data/models/mnist_net2_basic_0',
  'mode': 'base',
  'retry': {'max_retries': 3, 'delay': 5},
  'path': {'name': 'relative',
   'base_path_datasets': './data/datasets',
   'base_path_models': './data/models',
   'base_path': './data'},
  'model': {'name': 'net2',
   'object': {'_target_': 'modules.models.simple_net.Net',
    'num_layers': 2,
    'latent_dim': 128}},
  'training': {'name': 'basic',
   'device': 'cuda',
   'loss': {'_target_': 'torch.nn.NLLLoss'},
   'optimizer': {'_target_': 'torch.optim.SGD', 'lr': 0.1, 'momentum': 0.9},
   'scheduler': {'_target_': 'torch.optim.lr_scheduler.ReduceLROnPlateau',
    'mode': 'min',
    'factor': 0.1,
    'patience': 3},
   'epochs': 1,
   'gradient_accumulation_steps': 5,
   'loggers': {'tensorboard': {'_target_': 'lightning.pytorch.loggers.tensorboard.TensorBoardLogger',
     'save_dir': './data/mo

In [8]:
base_dir = Path("data\models")
results = load_folders(base_dir, max_pool=8)
df = pd.json_normalize(results)


Loading folders: 100%|██████████| 15/15 [00:08<00:00,  1.80it/s]


In [11]:
df.T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
_directory,data\models\mnist_net2_basic_0,data\models\mnist_net2_basic_1,data\models\mnist_net2_basic_2,data\models\mnist_net2_basic_3,data\models\mnist_net2_basic_4,data\models\mnist_net5_basic_0,data\models\mnist_net5_basic_1,data\models\mnist_net5_basic_2,data\models\mnist_net5_basic_3,data\models\mnist_net5_basic_4,data\models\mnist_net7_basic_0,data\models\mnist_net7_basic_1,data\models\mnist_net7_basic_2,data\models\mnist_net7_basic_3,data\models\mnist_net7_basic_4
csv.version_0.metrics,data\models\mnist_net2_basic_0\csv\version_0\m...,data\models\mnist_net2_basic_1\csv\version_0\m...,data\models\mnist_net2_basic_2\csv\version_0\m...,data\models\mnist_net2_basic_3\csv\version_0\m...,data\models\mnist_net2_basic_4\csv\version_0\m...,data\models\mnist_net5_basic_0\csv\version_0\m...,data\models\mnist_net5_basic_1\csv\version_0\m...,data\models\mnist_net5_basic_2\csv\version_0\m...,data\models\mnist_net5_basic_3\csv\version_0\m...,data\models\mnist_net5_basic_4\csv\version_0\m...,data\models\mnist_net7_basic_0\csv\version_0\m...,data\models\mnist_net7_basic_1\csv\version_0\m...,data\models\mnist_net7_basic_2\csv\version_0\m...,data\models\mnist_net7_basic_3\csv\version_0\m...,data\models\mnist_net7_basic_4\csv\version_0\m...
config.seed,0,1,2,3,4,0,1,2,3,4,0,1,2,3,4
config.suffix,,,,,,,,,,,,,,,
config.name,mnist_net2_basic_0,mnist_net2_basic_1,mnist_net2_basic_2,mnist_net2_basic_3,mnist_net2_basic_4,mnist_net5_basic_0,mnist_net5_basic_1,mnist_net5_basic_2,mnist_net5_basic_3,mnist_net5_basic_4,mnist_net7_basic_0,mnist_net7_basic_1,mnist_net7_basic_2,mnist_net7_basic_3,mnist_net7_basic_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
hydra_config.runtime.choices.hydra/launcher,NaN,NaN,NaN,NaN,NaN,basic,basic,NaN,basic,basic,basic,basic,basic,NaN,NaN
hydra_config.runtime.choices.hydra/output,NaN,NaN,NaN,NaN,NaN,default,default,NaN,default,default,default,default,default,NaN,NaN
hydra_config.verbose,NaN,NaN,NaN,NaN,NaN,False,False,NaN,False,False,False,False,False,NaN,NaN
config.data.dataloaders.train.multiprocessing_context,NaN,NaN,NaN,NaN,NaN,NaN,spawn,spawn,spawn,spawn,spawn,spawn,spawn,spawn,spawn


In [12]:
metrics_columns_list + agg_columns_list

['config.seed',
 'config.suffix',
 'config.name',
 'config.save_dir',
 'config.mode',
 'config.retry.max_retries',
 'config.retry.delay',
 'config.path.name',
 'config.path.base_path_datasets',
 'config.path.base_path_models',
 'config.path.base_path',
 'config.model.name',
 'config.model.object._target_',
 'config.model.object.num_layers',
 'config.model.object.latent_dim',
 'config.training.name',
 'config.training.device',
 'config.training.loss._target_',
 'config.training.optimizer._target_',
 'config.training.optimizer.lr',
 'config.training.optimizer.momentum',
 'config.training.scheduler._target_',
 'config.training.scheduler.mode',
 'config.training.scheduler.factor',
 'config.training.scheduler.patience',
 'config.training.epochs',
 'config.training.gradient_accumulation_steps',
 'config.training.loggers.tensorboard._target_',
 'config.training.loggers.tensorboard.save_dir',
 'config.training.loggers.tensorboard.name',
 'config.training.loggers.csv._target_',
 'config.trainin

In [17]:
metrics_columns = {
    **{c:c.replace("result.","") for c in df.columns if ("result." in c)}, 
    **{"run_info.total_time_seconds": "time"}}
agg_columns = {
    "config.data.name": "dataset",
    "config.model.name": "model"}

cols_renaming = {
    **metrics_columns,
    **agg_columns,}

metrics_columns_list = list(metrics_columns.values())
agg_columns_list = list(agg_columns.values())

# rename cols
dft = df.rename(columns=cols_renaming)
dft = dft[metrics_columns_list + agg_columns_list]

# Aggregate and scale
dft_mean = dft.groupby(agg_columns_list)[metrics_columns_list].mean().reset_index()
dft_std = dft.groupby(agg_columns_list)[metrics_columns_list].std().reset_index().fillna(0.001)
dft_std[dft_mean.isna()]=np.nan

# Merge mean and std
dft_merged = pd.merge(dft_mean, dft_std, on=agg_columns_list, suffixes=('_mean', '_std'))

# Create final dataframe with formatted text
dft_txt = dft_merged.copy()
for col in metrics_columns_list:
    mean_col = f"{col}_mean"
    std_col = f"{col}_std"
    dft_txt[col] = dft_merged.apply(
        lambda x: f"{x[mean_col]:.3f} ± {x[std_col]:.3f}" if not pd.isna(x[mean_col]) else "N/A",
        axis=1
    )
    dft_txt.drop(columns=[mean_col, std_col], inplace=True)



In [18]:
dft_txt

,dataset,model,train_loss,valid_loss,time
0,mnist,net2,1.673 ± 0.133,0.279 ± 0.095,46.927 ± 6.480
1,mnist,net5,2.619 ± 0.538,8.601 ± 14.109,50.473 ± 6.805
2,mnist,net7,2.302 ± 0.000,2.300 ± 0.000,134.659 ± 167.946
